# Experimental Design

Experimental design is the process in which we carry out research in an objective and controlled fashion. The purpose of this is to ensure we can make specific conclusions in reference to a hypothesis we have.

https://www.sciencedirect.com/topics/earth-and-planetary-sciences/experimental-design
